In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [57]:
%aiida
import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview

from apps.reactions.nebwork import NEBWorkchain

In [4]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

# Submit a NEB calculation

## Structure

In [32]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    # Fixed atoms as F
    try:
        bounds = map(int, text_fixed_atoms.value.split('..'))
        for a in range(bounds[0]-1, bounds[1]):
            atoms[a].symbol = 'F'
    except:
        pass
    
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

VBox(children=(StructureBrowser(children=(VBox(children=(VBox(children=(HTML(value=u'<p>Select the date range:</p>'), HBox(children=(Text(value=u'2018-03-05', description=u'From: ', style=DescriptionStyle(description_width=u'120px')), Text(value=u'2018-03-13', description=u'To: '))), Button(description=u'Search', layout=Layout(margin=u'1em 0 0 0'), style=ButtonStyle())), layout=Layout(border=u'1px solid #fafafa', padding=u'1em')), HTML(value=u'<hr>'), HBox(children=(RadioButtons(layout=Layout(width=u'25%'), options=('all', 'uploaded', 'edited', 'calculated'), value='all'), Dropdown(description=u'Process Label', layout=Layout(width=u'50%'), options=('All', u'FTGeoOptWorkChain', u'SlabGeoOptWorkChain', u'ReplicaWorkchain', u'EnergyWorkChain', u'STMWorkChain', u'DFTGeoOptWorkChain', u'NanoribbonWorkChain'), style=DescriptionStyle(description_width=u'120px'), value='All'))))), HTML(value=u'<hr>'), Dropdown(layout=Layout(width=u'900px'), options=OrderedDict([('Select a Structure (50 found)', False), (u'PK: 46811 | 2018-03-12 12:30 | C28H240Au896I2 | ff4-precursorI', <StructureData: uuid: 8408c6b9-3379-4d18-91ef-71631d26df1b (pk: 46811)>), (u'PK: 46796 | 2018-03-12 12:00 | C28H240Au896Br2 | ff3-precursorBr', <StructureData: uuid: 7fc19f6a-d5f1-455f-8615-4a580ff3d8de (pk: 46796)>), (u'PK: 46738 | 2018-03-12 09:41 | C28H242Au896 | ff2-precursor', <StructureData: uuid: 656ac4b8-f8cb-4d6d-aa25-1ae384194083 (pk: 46738)>), (u'PK: 46733 | 2018-03-12 09:40 | C42H242Au896 | ff1-hexabenzocoronene', <StructureData: uuid: c82f98e0-a19b-4016-a26f-076a6da257b1 (pk: 46733)>), (u'PK: 46728 | 2018-03-12 09:34 | C28H242Au1568 | ff2-precursor', <StructureData: uuid: 6b9257d5-04b8-4bb4-b344-70f92d3c0e55 (pk: 46728)>), (u'PK: 46723 | 2018-03-12 09:32 | C42H242Au1568 | ff1-hexabenzocoronene', <StructureData: uuid: 27d5f4ed-1608-49dc-8c7a-1b0e62f69b4e (pk: 46723)>), (u'PK: 12123 | 2018-03-11 07:30 | C18H236Au896 | ', <StructureData: uuid: 752e0403-e06c-423c-81c9-bd4297460823 (pk: 12123)>), (u'PK: 12113 | 2018-03-11 05:28 | C18H236Au896 | ', <StructureData: uuid: b4718fcb-0af9-40d4-a185-6e2145ececfb (pk: 12113)>), (u'PK: 12103 | 2018-03-10 23:47 | C18H236Au896 | ', <StructureData: uuid: b60009b9-530d-47cf-8ee4-b1c0b531a5e7 (pk: 12103)>), (u'PK: 12093 | 2018-03-10 08:50 | C18H236Au896 | ', <StructureData: uuid: caa473e7-21d7-4112-a583-59071b5be2db (pk: 12093)>), (u'PK: 12083 | 2018-03-09 08:13 | C18H236Au896 | ', <StructureData: uuid: e93592cd-dec6-44fe-bcd0-cbb4b3c699a1 (pk: 12083)>), (u'PK: 12073 | 2018-03-08 23:30 | C18H236Au896 | ', <StructureData: uuid: 8de72be0-782e-405f-bb53-9f65f50d978a (pk: 12073)>), (u'PK: 12070 | 2018-03-08 22:17 | C96H370Au1280 | ', <StructureData: uuid: 8871773a-2666-4254-9de8-035f578a0687 (pk: 12070)>), (u'PK: 12060 | 2018-03-08 21:50 | C96H370Au1280 | ', <StructureData: uuid: 27526ede-a0be-4a67-a106-14d74154e8aa (pk: 12060)>), (u'PK: 12050 | 2018-03-08 21:23 | C96H370Au1280 | ', <StructureData: uuid: 641604b8-d892-432c-9ec8-1df9f8801671 (pk: 12050)>), (u'PK: 12040 | 2018-03-08 20:31 | C96H370Au1280 | ', <StructureData: uuid: 74700c99-b813-4d47-a79a-cacb0fcc8926 (pk: 12040)>), (u'PK: 12030 | 2018-03-08 20:09 | C96H370Au1280 | ', <StructureData: uuid: 752a2c84-a8d0-4c7a-a826-5a14e4b74143 (pk: 12030)>), (u'PK: 12020 | 2018-03-08 19:06 | C96H370Au1280 | ', <StructureData: uuid: aacac070-45c3-4469-8392-53ce3278c5a0 (pk: 12020)>), (u'PK: 12010 | 2018-03-08 18:47 | C18H236Au896 | ', <StructureData: uuid: b5403299-10dc-4b75-9b4a-7903a80f57d6 (pk: 12010)>), (u'PK: 12000 | 2018-03-08 18:26 | C96H370Au1280 | ', <StructureData: uuid: 59a4591b-e519-4d19-8e13-37094571cef9 (pk: 12000)>), (u'PK: 11990 | 2018-03-08 18:09 | C96H370Au1280 | ', <StructureData: uuid: 885a50e4-8eee-4add-a4fa-8b14206d7d60 (pk: 11990)>), (u'PK: 11980 | 2018-03-08 17:45 | C96H370Au1280 | ', <StructureData: uuid: 35e3e32f-da1e-4006-a4b3-618756e377c2 (pk: 11980)>), (u'PK: 11970 | 2018-03-08 17:17 | C96H370Au1280 | ', <StructureData: uuid: 4c4d4ebd-dbdc-4373-8ef2-22c040

## Inputs

In [43]:
from collections import OrderedDict

# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})
qb.order_by({Code: {'id': 'asc'}})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    code = match[1]
    label = match[0]
    all_computers[label] = code

style = {'description_width': '160px'}
layout = {'width': '90%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

int_machines = ipw.IntText(value=1,
                           description='# Nodes per Replica',
                           style=style, layout=layout)

text_calc_name = ipw.Text(placeholder='Some cool name.',
                          description='Calculation Name',
                          style=style, layout=layout)

text_cell = ipw.Text(placeholder='40 40 40',
                     description='Cell (optional)',
                     style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..10',
                            description='Fixed Atoms',
                            style=style, layout={'width': '80%'})

btn_fixed_atoms = ipw.Button(description='apply (F)',
                             layout={'width': '10%'})

btn_fixed_atoms.on_click(lambda b: refresh_structure_view())

# Nodes * Processors/Node * Replicas
int_nproc_rep = ipw.IntText(description='Processors per replica',
                            value='8',
                            style=style, layout=layout)

int_replicas = ipw.IntSlider(description='Replicas',
                             value=7,
                             min=3, max=21,
                             style=style, layout=layout)

float_spring = ipw.FloatText(description='Spring constant',
                             value=0.05,
                             min=0.01, max=0.5, step=0.01,
                             style=style, layout=layout)

toggle_rotate = ipw.ToggleButton(description='Rotate frames',
                                 value=False,
                                 style=style, layout={'width': '45%'})

toggle_align = ipw.ToggleButton(description='Align frames',
                                value=False,
                                style=style, layout={'width': '45%'})

# [how many optimizations before climbing]
int_nstepsit = ipw.IntText(description='NSTEPS_IT',
                           value=1,
                           style=style, layout=layout)

toggle_endpoints = ipw.ToggleButton(description='Optimize endpoints',
                                    value=True,
                                    style=style, layout=layout)

display(
    drop_computer, int_machines,
    text_calc_name, text_cell,
    ipw.HBox([text_fixed_atoms, btn_fixed_atoms]),
    int_nproc_rep, int_replicas, float_spring,
    ipw.HBox([toggle_rotate, toggle_align]),
    int_nstepsit,
    toggle_endpoints
    
)

Dropdown(description=u'Computer', layout=Layout(width=u'90%'), options=OrderedDict([('Please select a computer', False), (u'daint-s746', <Code: Remote code 'cp2k_6.0_18265' on daint-s746, pk: 10942, uuid: 0bba26a3-e426-4572-b2df-59a874eb104c>), (u'marconi', <Code: Remote code 'cp2k_5.1_18268_marconi' on marconi, pk: 11314, uuid: af60eb34-5d27-4199-97f0-0c9d9a7235da>), (u'daint-s746-low', <Code: Remote code 'cp2k_6.0_18265' on daint-s746-low, pk: 46690, uuid: a5bd5c0e-bf85-408d-b562-39432cc926f8>)]), style=DescriptionStyle(description_width=u'160px'), value=False)

IntText(value=1, description=u'# Nodes per Replica', layout=Layout(width=u'90%'), style=DescriptionStyle(description_width=u'160px'))

Text(value=u'', description=u'Calculation Name', layout=Layout(width=u'90%'), placeholder=u'Some cool name.', style=DescriptionStyle(description_width=u'160px'))

Text(value=u'', description=u'Cell (optional)', layout=Layout(width=u'90%'), placeholder=u'40 40 40', style=DescriptionStyle(description_width=u'160px'))

HBox(children=(Text(value=u'', description=u'Fixed Atoms', layout=Layout(width=u'80%'), placeholder=u'1..10', style=DescriptionStyle(description_width=u'160px')), Button(description=u'apply (F)', layout=Layout(width=u'10%'), style=ButtonStyle())))

...

IntSlider(value=7, description=u'Replicas', layout=Layout(width=u'90%'), max=21, min=3, style=SliderStyle(description_width=u'160px'))

FloatText(value=0.05, description=u'Spring constant', layout=Layout(width=u'90%'), step=0.01, style=DescriptionStyle(description_width=u'160px'))

HBox(children=(ToggleButton(value=False, description=u'Rotate frames', layout=Layout(width=u'45%'), style=DescriptionStyle(description_width=u'160px')), ToggleButton(value=False, description=u'Align frames', layout=Layout(width=u'45%'), style=DescriptionStyle(description_width=u'160px'))))

IntText(value=1, description=u'NSTEPS_IT', layout=Layout(width=u'90%'), style=DescriptionStyle(description_width=u'160px'))

ToggleButton(value=True, description=u'Optimize endpoints', layout=Layout(width=u'90%'), style=DescriptionStyle(description_width=u'160px'))

In [56]:
toggle_dft = ipw.ToggleButtons(options=['Mixed DFTB', 'Mixed DFT', 'Full DFT'],
                               description='Calculation Type',
                               style=style, layout=layout)

display(toggle_dft)

ToggleButtons(description=u'Calculation Type', layout=Layout(width=u'90%'), options=('Mixed DFTB', 'Mixed DFT', 'Full DFT'), style=ToggleButtonsStyle(description_width=u'160px'), value='Mixed DFTB')

## Replicas
`Give the replica IDs in the right order initial->final.`

In [34]:
text_replica_pks = ipw.Text(placeholder='100 105 113 140',
                            description='Replica pks',
                            style=style, layout=layout)

display(text_replica_pks)

Text(value=u'', description=u'Replica pks', layout=Layout(width=u'80%'), placeholder=u'100 105 113 140', style=DescriptionStyle(description_width=u'160px'))

## Submission

In [35]:
def on_submit(b):
    cp2k_code = drop_computer.value
    struct = struct_browser.results.value
    num_machines = Int(int_machines.value)
    calc_name = Str(text_replica_name.value)
    cell = Str(text_cell.value)
    fixed_atoms = Str(text_fixed_atoms.value)

    colvar_targets = Str(text_colvar_targets.value)
    spring = Float(float_spring.value)

    outputs = submit(NEBWorkchain,
                     cp2k_code=cp2k_code,
                     structure=struct,
                     num_machines=num_machines,
                     replica_name=replica_name,
                     cell=cell,
                     fixed_atoms=fixed_atoms,
                     colvar_targets=colvar_targets,
                     target_unit=Str(target_unit),
                     spring=spring,
                     spring_unit=Str(spring_unit),
                     subsys_colvar=generate_cp2k_subsys_colvar(),
                     calc_type=calc_type)

    print(outputs)
    the_workfunction = load_node(outputs.pid)
    the_workfunction.description = replica_name

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

Button(description=u'Submit', style=ButtonStyle())

Output()